# AutoGen Основен Пример

В този примерен код ще използвате [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework, за да създадете основен агент.

Целта на този пример е да ви покаже стъпките, които по-късно ще използваме в допълнителните примерни кодове при реализирането на различни агентски модели.


## Импортирайте необходимите Python пакети


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Създаване на клиента

В този пример ще използваме [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) за достъп до LLM.

`model` е дефиниран като `gpt-4o-mini`. Опитайте да промените модела на друг, наличен в GitHub Models marketplace, за да видите различни резултати.

Като бърз тест, просто ще изпълним прост въпрос - `Каква е столицата на Франция`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Определяне на Агента

След като настроихме `client` и потвърдихме, че работи, нека създадем `AssistantAgent`. Всеки агент може да бъде назначен с:  
**name** - Кратко име, което ще бъде полезно за препратки в потоци с множество агенти.  
**model_client** - Клиентът, който създадохте в предишната стъпка.  
**tools** - Налични инструменти, които агентът може да използва за изпълнение на задача.  
**system_message** - Метаподсказка, която определя задачата, поведението и тона на LLM.  

Можете да промените системното съобщение, за да видите как LLM реагира. Ще разгледаме `tools` в Урок №4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Стартиране на Агента

Следната функция ще стартира агента. Използваме метода `on_message`, за да актуализираме състоянието на Агента с новото съобщение.

В този случай актуализираме състоянието с ново съобщение от потребителя, което е `"Планирай ми страхотна слънчева ваканция"`.

Можете да промените съдържанието на съобщението, за да видите как LLM реагира по различен начин.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
